In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 1
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 2
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:24.0000 R:24.0000 rate:0.0480 aloss:0.6981 eloss:3.2821 aloss2:2.3692 exploreP:0.9976
Episode:1 meanR:20.0000 R:16.0000 rate:0.0320 aloss:0.6950 eloss:3.2749 aloss2:2.3642 exploreP:0.9960
Episode:2 meanR:23.0000 R:29.0000 rate:0.0580 aloss:0.6934 eloss:3.2787 aloss2:2.3563 exploreP:0.9932
Episode:3 meanR:20.7500 R:14.0000 rate:0.0280 aloss:0.6926 eloss:3.2936 aloss2:2.3467 exploreP:0.9918
Episode:4 meanR:20.2000 R:18.0000 rate:0.0360 aloss:0.6916 eloss:3.2606 aloss2:2.3387 exploreP:0.9901
Episode:5 meanR:19.1667 R:14.0000 rate:0.0280 aloss:0.6873 eloss:3.2585 aloss2:2.3202 exploreP:0.9887
Episode:6 meanR:22.0000 R:39.0000 rate:0.0780 aloss:0.6928 eloss:3.2492 aloss2:2.3334 exploreP:0.9849
Episode:7 meanR:22.7500 R:28.0000 rate:0.0560 aloss:0.6914 eloss:3.2344 aloss2:2.3184 exploreP:0.9821
Episode:8 meanR:22.6667 R:22.0000 rate:0.0440 aloss:0.6898 eloss:3.2254 aloss2:2.3113 exploreP:0.9800
Episode:9 meanR:21.9000 R:15.0000 rate:0.0300 aloss:0.6921 eloss:3.2128 aloss2:2.2

Episode:80 meanR:29.4321 R:52.0000 rate:0.1040 aloss:0.6875 eloss:3.4958 aloss2:2.3462 exploreP:0.7900
Episode:81 meanR:29.6341 R:46.0000 rate:0.0920 aloss:0.7251 eloss:3.5169 aloss2:2.3322 exploreP:0.7864
Episode:82 meanR:30.4217 R:95.0000 rate:0.1900 aloss:0.6887 eloss:3.5294 aloss2:2.3735 exploreP:0.7791
Episode:83 meanR:31.0119 R:80.0000 rate:0.1600 aloss:0.6903 eloss:3.5572 aloss2:2.3832 exploreP:0.7730
Episode:84 meanR:31.3294 R:58.0000 rate:0.1160 aloss:0.6919 eloss:3.6265 aloss2:2.3668 exploreP:0.7685
Episode:85 meanR:31.8605 R:77.0000 rate:0.1540 aloss:0.7053 eloss:3.5972 aloss2:2.4036 exploreP:0.7627
Episode:86 meanR:32.7816 R:112.0000 rate:0.2240 aloss:0.7019 eloss:3.6163 aloss2:2.3968 exploreP:0.7543
Episode:87 meanR:32.8182 R:36.0000 rate:0.0720 aloss:0.6961 eloss:3.6507 aloss2:2.4173 exploreP:0.7517
Episode:88 meanR:32.7753 R:29.0000 rate:0.0580 aloss:0.7393 eloss:3.6675 aloss2:2.3764 exploreP:0.7495
Episode:89 meanR:33.0111 R:54.0000 rate:0.1080 aloss:0.7013 eloss:3.5939

Episode:159 meanR:53.1800 R:45.0000 rate:0.0900 aloss:0.6922 eloss:3.2018 aloss2:2.4737 exploreP:0.5017
Episode:160 meanR:53.6600 R:68.0000 rate:0.1360 aloss:0.6846 eloss:3.1741 aloss2:2.4836 exploreP:0.4983
Episode:161 meanR:53.3200 R:26.0000 rate:0.0520 aloss:0.6855 eloss:3.1877 aloss2:2.4680 exploreP:0.4971
Episode:162 meanR:52.8300 R:49.0000 rate:0.0980 aloss:0.6885 eloss:3.1958 aloss2:2.4499 exploreP:0.4947
Episode:163 meanR:52.6300 R:40.0000 rate:0.0800 aloss:0.6843 eloss:3.1688 aloss2:2.4666 exploreP:0.4928
Episode:164 meanR:53.2500 R:89.0000 rate:0.1780 aloss:0.6798 eloss:3.1682 aloss2:2.4550 exploreP:0.4885
Episode:165 meanR:54.1500 R:108.0000 rate:0.2160 aloss:0.6900 eloss:3.1757 aloss2:2.4386 exploreP:0.4833
Episode:166 meanR:55.4700 R:183.0000 rate:0.3660 aloss:0.6933 eloss:3.1614 aloss2:2.4111 exploreP:0.4748
Episode:167 meanR:57.7700 R:245.0000 rate:0.4900 aloss:0.6880 eloss:3.1504 aloss2:2.3771 exploreP:0.4635
Episode:168 meanR:58.2000 R:72.0000 rate:0.1440 aloss:0.6868 

Episode:237 meanR:179.7300 R:93.0000 rate:0.1860 aloss:0.7908 eloss:3.2661 aloss2:2.3302 exploreP:0.1067
Episode:238 meanR:179.7000 R:53.0000 rate:0.1060 aloss:0.8195 eloss:3.2608 aloss2:2.3169 exploreP:0.1062
Episode:239 meanR:179.7600 R:29.0000 rate:0.0580 aloss:0.7915 eloss:3.2821 aloss2:2.3338 exploreP:0.1059
Episode:240 meanR:179.3600 R:42.0000 rate:0.0840 aloss:0.7937 eloss:3.3309 aloss2:2.3511 exploreP:0.1055
Episode:241 meanR:179.0900 R:51.0000 rate:0.1020 aloss:0.7918 eloss:3.2986 aloss2:2.3606 exploreP:0.1050
Episode:242 meanR:179.8100 R:99.0000 rate:0.1980 aloss:0.7959 eloss:3.2879 aloss2:2.3587 exploreP:0.1041
Episode:243 meanR:178.9900 R:46.0000 rate:0.0920 aloss:0.8622 eloss:3.3021 aloss2:2.4309 exploreP:0.1037
Episode:244 meanR:179.2600 R:45.0000 rate:0.0900 aloss:0.7831 eloss:3.2765 aloss2:2.4100 exploreP:0.1032
Episode:245 meanR:178.1900 R:30.0000 rate:0.0600 aloss:0.8132 eloss:3.2792 aloss2:2.3754 exploreP:0.1030
Episode:246 meanR:178.0100 R:114.0000 rate:0.2280 aloss

Episode:315 meanR:121.7900 R:63.0000 rate:0.1260 aloss:0.7077 eloss:3.2292 aloss2:2.2901 exploreP:0.0538
Episode:316 meanR:119.4800 R:82.0000 rate:0.1640 aloss:0.6995 eloss:3.2455 aloss2:2.2457 exploreP:0.0534
Episode:317 meanR:119.8900 R:193.0000 rate:0.3860 aloss:0.7505 eloss:3.2395 aloss2:2.1854 exploreP:0.0526
Episode:318 meanR:118.8100 R:115.0000 rate:0.2300 aloss:0.7175 eloss:3.2320 aloss2:2.1833 exploreP:0.0521
Episode:319 meanR:117.5500 R:122.0000 rate:0.2440 aloss:0.7249 eloss:3.2230 aloss2:2.1917 exploreP:0.0516
Episode:320 meanR:116.4900 R:109.0000 rate:0.2180 aloss:0.7372 eloss:3.2002 aloss2:2.2062 exploreP:0.0512
Episode:321 meanR:115.3400 R:113.0000 rate:0.2260 aloss:0.7275 eloss:3.1566 aloss2:2.2098 exploreP:0.0507
Episode:322 meanR:113.6400 R:125.0000 rate:0.2500 aloss:0.7307 eloss:3.1929 aloss2:2.2231 exploreP:0.0502
Episode:323 meanR:112.4300 R:118.0000 rate:0.2360 aloss:0.7286 eloss:3.1647 aloss2:2.2212 exploreP:0.0497
Episode:324 meanR:112.9600 R:163.0000 rate:0.326

Episode:393 meanR:128.9700 R:306.0000 rate:0.6120 aloss:0.6912 eloss:3.2197 aloss2:2.2602 exploreP:0.0249
Episode:394 meanR:131.4200 R:321.0000 rate:0.6420 aloss:0.6969 eloss:3.2306 aloss2:2.2556 exploreP:0.0244
Episode:395 meanR:133.5000 R:300.0000 rate:0.6000 aloss:0.6999 eloss:3.2081 aloss2:2.2495 exploreP:0.0240
Episode:396 meanR:135.7000 R:319.0000 rate:0.6380 aloss:0.6891 eloss:3.2294 aloss2:2.2581 exploreP:0.0236
Episode:397 meanR:138.8800 R:404.0000 rate:0.8080 aloss:0.6990 eloss:3.2176 aloss2:2.2460 exploreP:0.0230
Episode:398 meanR:142.4600 R:450.0000 rate:0.9000 aloss:0.6934 eloss:3.2122 aloss2:2.2637 exploreP:0.0224
Episode:399 meanR:144.3900 R:382.0000 rate:0.7640 aloss:0.6879 eloss:3.2178 aloss2:2.2605 exploreP:0.0220
Episode:400 meanR:144.5300 R:136.0000 rate:0.2720 aloss:0.6998 eloss:3.2050 aloss2:2.2510 exploreP:0.0218
Episode:401 meanR:145.3600 R:194.0000 rate:0.3880 aloss:0.6882 eloss:3.2086 aloss2:2.2620 exploreP:0.0216
Episode:402 meanR:147.6000 R:369.0000 rate:0.7

Episode:471 meanR:171.8600 R:155.0000 rate:0.3100 aloss:0.6545 eloss:3.2001 aloss2:2.2486 exploreP:0.0144
Episode:472 meanR:170.9500 R:115.0000 rate:0.2300 aloss:0.6567 eloss:3.2009 aloss2:2.2484 exploreP:0.0143
Episode:473 meanR:168.6500 R:105.0000 rate:0.2100 aloss:0.6567 eloss:3.2015 aloss2:2.2543 exploreP:0.0143
Episode:474 meanR:169.3700 R:142.0000 rate:0.2840 aloss:0.6448 eloss:3.1983 aloss2:2.2841 exploreP:0.0142
Episode:475 meanR:167.3100 R:133.0000 rate:0.2660 aloss:0.6204 eloss:3.2194 aloss2:2.2767 exploreP:0.0141
Episode:476 meanR:166.8000 R:134.0000 rate:0.2680 aloss:0.6550 eloss:3.1969 aloss2:2.2482 exploreP:0.0141
Episode:477 meanR:166.8600 R:123.0000 rate:0.2460 aloss:0.6655 eloss:3.2001 aloss2:2.2474 exploreP:0.0140
Episode:478 meanR:165.1400 R:108.0000 rate:0.2160 aloss:0.6761 eloss:3.2166 aloss2:2.2432 exploreP:0.0140
Episode:479 meanR:164.6700 R:96.0000 rate:0.1920 aloss:0.6540 eloss:3.2086 aloss2:2.2660 exploreP:0.0140
Episode:480 meanR:164.5000 R:127.0000 rate:0.25

Episode:549 meanR:135.2100 R:149.0000 rate:0.2980 aloss:0.6162 eloss:3.1992 aloss2:2.2832 exploreP:0.0115
Episode:550 meanR:135.6900 R:161.0000 rate:0.3220 aloss:0.5958 eloss:3.2000 aloss2:2.2968 exploreP:0.0115
Episode:551 meanR:135.8700 R:117.0000 rate:0.2340 aloss:0.6371 eloss:3.1857 aloss2:2.2724 exploreP:0.0115
Episode:552 meanR:134.7000 R:139.0000 rate:0.2780 aloss:0.5887 eloss:3.2041 aloss2:2.2959 exploreP:0.0115
Episode:553 meanR:135.8000 R:216.0000 rate:0.4320 aloss:0.5894 eloss:3.2105 aloss2:2.2837 exploreP:0.0115
Episode:554 meanR:134.6300 R:117.0000 rate:0.2340 aloss:0.5765 eloss:3.2058 aloss2:2.2779 exploreP:0.0114
Episode:555 meanR:135.0000 R:163.0000 rate:0.3260 aloss:0.6034 eloss:3.1889 aloss2:2.2794 exploreP:0.0114
Episode:556 meanR:135.3900 R:134.0000 rate:0.2680 aloss:0.6071 eloss:3.1963 aloss2:2.2949 exploreP:0.0114
Episode:557 meanR:134.4400 R:142.0000 rate:0.2840 aloss:0.6148 eloss:3.1984 aloss2:2.2827 exploreP:0.0114
Episode:558 meanR:134.6000 R:148.0000 rate:0.2

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
